In [1]:
import os
import yaml
import cosmo
import numpy
import itertools
from glob import glob
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from cosmo.monitors.data_models import NUVDarkDataModel
from cosmo.filesystem import find_files, data_from_exposures, data_from_jitters
from cosmo.monitor_helpers import absolute_time, explode_df
from monitorframe.datamodel import BaseDataModel
from monitorframe.monitor import BaseMonitor

In [2]:
with open(os.environ['COSMO_CONFIG']) as yamlfile:
    SETTINGS = yaml.safe_load(yamlfile)

FILES_SOURCE = SETTINGS['filesystem']['source']
COS_MONITORING = SETTINGS['output']

In [4]:
class NUVDarkDataModel(BaseDataModel):
    cosmo_layout = False
    # 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17
#   program_ids = ['15776/', '15538/', '14942/', '14521/', '14442/', '13974/', 
#                        '13528/', '13126/', '12720/', '12420/', '11894/']
    program_ids = ['15538/']
    # subdir_pattern = '?????'
    # ^ this will need to be added back once we have an all corrtag model

    def get_new_data(self):
        header_request = {
            0: ['ROOTNAME'],
            1: ['EXPTIME', 'EXPSTART']
        }

        table_request = {
            1: ['XCORR', 'YCORR', 'TIME'],
            3: ['TIME', 'LATITUDE', 'LONGITUDE']
        }

        files = []

        for prog_id in self.program_ids:
            new_files_source = os.path.join(FILES_SOURCE, prog_id)
            files += find_files('*corrtag*', data_dir=new_files_source)

        if self.model is not None:
            currently_ingested = [item.FILENAME for item in self.model.select(self.model.FILENAME)]

            for file in currently_ingested:
                files.remove(file)

        if not files:   # No new files
            return pd.DataFrame()

        data_results = data_from_exposures(
            files,
            header_request=header_request,
            table_request=table_request
        )

        return data_results

In [5]:
nuv_model = NUVDarkDataModel()
data_results = nuv_model.get_new_data()
data_results

""
